## Web Scraping Indeed and classifying Data Science Jobs


In this project, I will be looking into building a job classifier. This project has 4 main parts:

1. web scraping the Indded website for data science job postings
2. bulid a classifier to classify the differnt data science types/levels
3. classify a real data scientist based upon skills, experience etc
4. provide a list of skills required for a data scientist to move one level up


The first part (web scraping) is based on the tutroail at https://github.com/aakashtandel/Web-Scraping-Indeed/blob/master/Code/Project%203%20-%20Web%20Scraping%20Indeed%20Job%20Listings%20Jupyter%20Notebook.ipynb

### 1. Scraping job listings from https://www.indeed.co.uk/

We will be scraping listings from indeed.co.uk using BeautifulSoup.

In [15]:
url = "https://www.indeed.co.uk/jobs?q=data%20scientist&l=Bristol&start=10"

In [16]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [17]:
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")

In [18]:
results_ = soup.find_all(class_= "result")
len(results_)

10

View a single result

In [19]:
results_[4]

<div class="jobsearch-SerpJobCard unifiedRow row result" data-jk="fbffdf3b8f6b438c" data-tn-component="organicJob" id="p_fbffdf3b8f6b438c">
<h2 class="title">
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/rc/clk?jk=fbffdf3b8f6b438c&amp;fccid=b6893c92ad39abbc&amp;vjs=3" id="jl_fbffdf3b8f6b438c" onclick="setRefineByCookie([]); return rclk(this,jobmap[4],true,0);" onmousedown="return rclk(this,jobmap[4],0);" rel="noopener nofollow" target="_blank" title="Machine Learning Engineer">
Machine Learning Engineer</a>
</h2>
<div class="sjcl">
<div>
<span class="company">
Ultraleap Ltd.</span>
</div>
<div class="recJobLoc" data-rc-loc="Bristol" id="recJobLoc_fbffdf3b8f6b438c" style="display: none"></div>
<span class="location accessible-contrast-color-location">Bristol</span>
</div>
<table class="jobCardShelfContainer"><tr class="jobCardShelf"><td class="jobCardShelfItem indeedApply"><span class="jobCardShelfIcon"><svg fill="none" height="16" viewbox="0 0 20 20" width="16"><

For our classifier we need: 
1. the job title: which is an attribute in the jobtitle class
2. the description: which can be accessed through the href link in the jobtitle class

Other information that may be important, but not necessarily are;
1. the location
2. company name
3. salary

let's find all 5 in our example

In [20]:
ti_ = results_[4].find(class_='jobtitle')
ti_

<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/rc/clk?jk=fbffdf3b8f6b438c&amp;fccid=b6893c92ad39abbc&amp;vjs=3" id="jl_fbffdf3b8f6b438c" onclick="setRefineByCookie([]); return rclk(this,jobmap[4],true,0);" onmousedown="return rclk(this,jobmap[4],0);" rel="noopener nofollow" target="_blank" title="Machine Learning Engineer">
Machine Learning Engineer</a>

In [21]:
ti_.attrs

{'target': '_blank',
 'id': 'jl_fbffdf3b8f6b438c',
 'href': '/rc/clk?jk=fbffdf3b8f6b438c&fccid=b6893c92ad39abbc&vjs=3',
 'onmousedown': 'return rclk(this,jobmap[4],0);',
 'onclick': 'setRefineByCookie([]); return rclk(this,jobmap[4],true,0);',
 'rel': ['noopener', 'nofollow'],
 'title': 'Machine Learning Engineer',
 'class': ['jobtitle', 'turnstileLink'],
 'data-tn-element': 'jobTitle'}

In [22]:
#get the title
ti_.attrs["title"]


'Machine Learning Engineer'

In [23]:
#get the description
#to get teh descrition we will need to extract the data from the url
# it exists in class="jobsearch-jobDescriptionText"

URL_ = "https://www.indeed.co.uk"+ti_.attrs["href"]
html_ = requests.get(URL_)
soup_ = BeautifulSoup(html_.content, 'html.parser', from_encoding="utf-8")
soup_.find(class_= "jobsearch-jobDescriptionText").text.strip()

'We are looking for an experienced and motivated top-level Machine Learning Engineer for our spatial tracking team in the UK which is focused on developing new ways for humans and machines to interact.\n\nResponsibilities\n\nResearch, code and deliver alongside our product and engineering team to test and release new hand-tracking features to our users worldwide;\nApply mathematical techniques (geometry, linear algebra, numerical methods), machine learning and artificial intelligence techniques to improve spatial tracking;\nOptimise models to satisfy performance criteria, including mapping to software on various platforms;\nParticipate in high quality processes including reviews, test, verification, and process improvement.\n\nExperience and Skill-Set\n\nAn MSc or PhD in a quantitative field of STEM, Computer Science or Engineering with a focus on Machine Learning or industry experience in Machine Learning related work;\nStrong experience in applied mathematics, algorithm design and pe

Now merge with the parse function from https://github.com/aakashtandel/Web-Scraping-Indeed/blob/master/Code/Project%203%20-%20Web%20Scraping%20Indeed%20Job%20Listings%20Jupyter%20Notebook.ipynb 

In [24]:
def parse(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Description"])
    for each in soup.find_all(class_= "result" ):
        try: 
            title = each.find(class_='jobtitle').text.replace('\n', '')
        except:
            title = 'None'
        try:
            location = each.find('span', {'class':"location" }).text.replace('\n', '')
        except:
            location = 'None'
        try: 
            company = each.find(class_='company').text.replace('\n', '')
        except:
            company = 'None'
        try:
            salary = each.find('span', {'class':'no-wrap'}).text
        except:
            salary = 'None'
        
        url_ = "https://www.indeed.co.uk" + each.find(class_='jobtitle').attrs["href"]
        html_ = requests.get(url_)
        soup_ = BeautifulSoup(html_.content, 'html.parser', from_encoding="utf-8")
        description = soup_.find(class_= "jobsearch-jobDescriptionText").text.strip()
        
        df = df.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Description':description}, ignore_index=True)
    return df

In [25]:
parse(url)

,Title,Location,Company,Salary,Description
0,Junior/Mid-Level Data Analyst or Scientist,Bristol,Heat Recruitment,"\n\n£25,000 - £40,000 a year\n",I’m very pleased to be working with a superb B...
1,Data Scientist,Bristol,Prolancer,"\n\n£50,000 - £65,000 a year\n",One of the world’s leading financial consultat...
2,Senior Data Scientist - Electrical Power Systems,Bristol,Toumetis,None,TouMetis Job Description\n\nSenior Data Scient...
3,Research Associate or Senior Research Associat...,Bristol BS8,University of Bristol,"\n\n£33,797 - £38,017 a year\n",We have an exciting opportunity for a talented...
4,Machine Learning Engineer,Bristol,Ultraleap Ltd.,None,We are looking for an experienced and motivate...
5,Senior Data Scientist - Bristol University Team,Bristol,LV=Careers,None,Location\nBristol\nRef\nGI007058\nClosing date...
6,"Software Engineer (C++/Python), AI/Machine Lea...",Bristol,Linux Recruit,"\n\n£45,000 - £70,000 a year\n","Artificial Intelligence, or simply 'AI', is on..."
7,Graduate Data Scientist,Newport,Concept Resourcing,\n\n£140 - £145 a day\n,Are you a graduate in a numerical subject seek...
8,Machine Learning,Bristol,Cubiq Recruitment,"\n\n£30,000 - £75,000 a year\n",Description\nMachine Learning Engineer (Reinfo...
9,Data Scientist,Newport,Concept Resourcing,\n\n£400 - £500 a day\n,My client are a prestigious government organis...


### scale up

Now that we have our function to extract the raw data, we need to expand our search method. We can do so by including a range of cities and adding a limit to the number of results. The method is outlined in the notebook liked to the top of this page. Here I will simply copy and edit to my particular case.

In [41]:
url_template = "https://www.indeed.co.uk/jobs?q=data%20scientist&l={}&start={}"
max_results_per_city = 2000 # set to a high value for more results

cities = set(["Bristol","London","Leeds","Manchester","Edinburgh","Birmingham","Liverpool","Cardiff"]) # will use a smaller sample for now
#cities = set(["Bristol"])

i = 0
results=[]
df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Description"])
for city in cities:
    for start in range(0,max_results_per_city, 10): # iterate in steps of 10
        url = url_template.format(city, start)
        df_temp = parse(url)
        df = df.append(df_temp, ignore_index=True)

        # save the result as csv file
df.to_csv('Indeed_project_uncleaned.csv', encoding='utf-8')

In [42]:
df.head()

,Title,Location,Company,Salary,Description
0,Product Owner - up to £90k - Manchester -Pytho...,None,Circle Recruitment,"\n\n£70,000 - £90,000 a year\n","Product Owner-up to £90k-Manchester-Python, AI..."
1,Data Scientist,Manchester,QBE,None,Data Scientist\nManchester/Leeds\nWho we are\n...
2,Data Scientist,Wilmslow,Warner Bros. Entertainment Group,None,"Opportunity Overview\nPlaydemic, a division of..."
3,Senior Data Scientist,Manchester,Booking.com,None,Job title: Senior Data Scientist\n\nLocation: ...
4,Ecommerce Data Scientist,Warrington,The Candidate,None,Ecommerce Data Scientist – Ecommerce – Warring...


In [43]:
df.shape

(18044, 5)